## Теоретическая часть

1. Вспомним прошлый вебинар, мы рассматривали User-User рекомендации и Item-Item рекомендации. Чем они отличаются и чем они похожи? Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?  
В чем принципиальные отличия item-item рекомендаций от ALS?


item-item recommender - это аналог алгоритма KNN, то есть алгоритма ближайших соседей, но модифицированный на очень большие расстояния между объектами. ALS - этот алгоритм находит приближенные разложения изначальной - разреженной матрицы, вместо того, что бы искать объекты в этой матрице напрямую, по сути уменьшает размерность изначальной матрицы разбивая ее на 2 более узкие в которой количество взаимодействий и не взаимодействий 50/50%. 

2. Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров 


заимодействие пользователя с контентом в соц сетях: нравится - не нравится, репост - или нет, коммент - или нет. Факт покупки товара: купил первый товар или второй, добавил в корзину первый товар или второй. Просмотр страничек или посещение сайтов переходы по ссылкам: прошел по ссылке на сайт или нет. Фильтрация писем: добавил письмо в спам или нет.

3. Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)


Считает лишь факт взаимодействия пользователя с товаром, но веса зависят от количества взаимодействий. После уменьшения размерности матрици на 2 более узких, обратно целую матрицу восстановить уже нельзя.

## Практическая часть


In [38]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys
from IPython.display import clear_output
from tqdm import tqdm 

tqdm.pandas()

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)




In [39]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [40]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [41]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

/home/ivan/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


### Подбор матрицы $c_{ui}$
Попробуйте различные варианты матрицы весов (3+ вариантов). Обучите алгоритм для различных $C$. В качестве результата приведите таблицу: матрица весов - результат на train и validation.
Сделате качественные выводы.


In [42]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [43]:
def als_recomendation(values='quantity', aggfunc='count', factors=100, regularization=0.001): 
    from pandas.core.frame import Series
    user_item_matrix = pd.pivot_table(data_train, 
                                    index='user_id', 
                                    columns='item_id', 
                                    values= values, # Можно пробоват  и другие варианты
                                    aggfunc=aggfunc, # Проверим разные варианты агрегирующих функций.  
                                    fill_value=0
                                    )

    user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

    # переведем в формат saprse matrix
    sparse_user_item = csr_matrix(user_item_matrix).tocsr()
    # user_item_matrix.head(3)
    userids = user_item_matrix.index.values
    itemids = user_item_matrix.columns.values

    matrix_userids = np.arange(len(userids))
    matrix_itemids = np.arange(len(itemids))

    id_to_itemid = dict(zip(matrix_itemids, itemids))
    id_to_userid = dict(zip(matrix_userids, userids))

    itemid_to_id = dict(zip(itemids, matrix_itemids))
    userid_to_id = dict(zip(userids, matrix_userids))

    model = AlternatingLeastSquares(factors=factors, 
                                    regularization=regularization,
                                    iterations=15, 
                                    calculate_training_loss=True, 
                                    num_threads=4)

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=True)

    def get_recommendations(user, model, N=5):
        res = [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[user], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=N, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)]
        return res
        
    result['als'] = result['user_id'].progress_apply(lambda x: get_recommendations(x, model=model, N=5))
    result_precission =  result.progress_apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
    clear_output(wait=True) # Очищаем поле вывода. 
    return result_precission

In [44]:
results_dic = { 'values':[], 'aggfunc':[], 'precission':[]} 

for values in ['quantity','sales_value','store_id']:
    for aggfunc in ['sum', 'count','mean']:
        results_dic['values'].append(values)
        results_dic['aggfunc'].append(aggfunc)
        results_dic['precission'].append(als_recomendation(values=values, aggfunc=aggfunc))
        
pd.DataFrame(results_dic)

,values,aggfunc,precission
0,quantity,sum,0.144858
1,quantity,count,0.159256
2,quantity,mean,0.166601
3,sales_value,sum,0.098139
4,sales_value,count,0.159647
5,sales_value,mean,0.130950
6,store_id,sum,0.007640
7,store_id,count,0.158374
8,store_id,mean,0.007640


### Оптимизация гипперпараметров
Для лучшей матрицы весов из первого задания подберите оптимальные $\lambda$ и n_factors. Подбор можно делать вручную (цикл в цикле, аналог sklearn.GridSearch, или случайно - sklearn.RandomSearch). Или Вы можете воспользоваться библиотеками для автоматического подбора гипперпараметров (любые на Ваш вкус). В качестве результата постройте графики:
1. Значение параметра - время обучения 
2. Значение параметра - качество train, качество validation  

Сделайте качественные выводы

In [45]:
%%time
results_dic = { 'factors':[], 'lambda':[], 'precission':[]} 

for factors in [10, 30, 50, 100, 150, 200, 300]:
    for regularization in [0.1, 0.01, 0.001, 0.0001]:
        results_dic['factors'].append(factors)
        results_dic['lambda'].append(regularization)
        results_dic['precission'].append(als_recomendation(values='quantity',
                                                           aggfunc='mean',
                                                           factors=factors,
                                                           regularization=regularization))
        
result_df=pd.DataFrame(results_dic)
result_df

CPU times: user 2h 53min 57s, sys: 7h 9s, total: 9h 54min 6s
Wall time: 26min 27s


,factors,lambda,precission
0,10,0.1000,0.154750
1,10,0.0100,0.150833
2,10,0.0010,0.151714
3,10,0.0001,0.149167
4,30,0.1000,0.163075
5,30,0.0100,0.160039
6,30,0.0010,0.163369
7,30,0.0001,0.160921
8,50,0.1000,0.166895
9,50,0.0100,0.168168


По всей видимости большое количество малозначительных факторов засоряют данные и не несут предсказательной способности